<a href="https://colab.research.google.com/github/nekMont/Machine-Learning/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook we're going to cover some of the most fundamental concepts of tensors.

More specifically, we are going cover: 
* Introduction to tensors 
* Getting information from tensors 
* manipulating tensors 
* tensors & numpy 
* using @tf.function (a way to speed up your regular python functions) 
* using GPUs with tensorflow (or TPUs) 
*exercises to try for yourself 

##Introduction to tensors

In [ ]:
#Import TensorFlow 
import tensorflow as tf 
print(tf.__version__) 

2.9.2


In [ ]:
#creating tensors with tf.constant()
scalar = tf.constant(7) 
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim 

0

In [ ]:
# Create a vector 
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# check the dimension of our  vector 
vector.ndim

1

In [ ]:
# create a matrix (has more than one dimension) 
matrix = tf.constant([[10,7], 
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
matrix.ndim

2

In [ ]:
# creat another matrix 
another_matrix = tf.constant([[10.,7.],
                              [3.,2.],
                             [8.,9.]], dtype=tf.float16)#specify the data type
another_matrix                  

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
#what is the number of dimensions of another matrix
another_matrix.ndim

2

In [ ]:
#lets create a tensor 
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                      [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
tensor.ndim

3

what we've createed so far: 

* Scalar: a single number 
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers 
* Tensor: n-dimensional arrayy of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)  )

In [ ]:
test = tf.constant([[5,2,3,2],
                    [2,3,4,5],
                    [2,302,2,1]])
test

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[  5,   2,   3,   2],
       [  2,   3,   4,   5],
       [  2, 302,   2,   1]], dtype=int32)>

In [ ]:
test.ndim

2


Note: in test I was trying to see if the number of columns would change our dimension 
to 4, however this wasn't the case. Just because there are 4 values in column space does not mean
that ndim is equal to 4. 
  In the tensor example there are 3 brackets to indicate that the the tensor is three dimentsional
  this I may have to research more on because those brackets are super confusing.

### Creating tensors with `tf.variable` 

In [ ]:
# create the same tensor with tf.Variable() as above 
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Let's try to change one of the elements in our changeable tensor 
changeable_tensor[0] = 7 

TypeError: ignored

In [ ]:
# how about we try and .assign() 
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# Lets try to change our unchangeable tensor 
unchangeable_tensor[0].assign(7) 

AttributeError: ignored

**Note:** very rarely in practice will yoou need to decide whether to use `tf.constant` or `tf.variable` to create tensors, as TensorFlow does this 
for you. However, if in double, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors or some orbitrary size which contain random numbers 

In [ ]:
# Create two randome but the same tensors 
                                        #x
random_1 = tf.random.Generator.from_seed(7) #set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape = (3,2))
#normal distrubtion 
#uniform distrubion 

#are they equal? 
random_1,random_2, random_1 == random_2

#these are psuedo random
#setting the seed means, create some randoms numbers but flavor them with x, whe
#x is the seed.

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

Note: When a neural network starts to learn, it starts with random patterns. 
then slowly adjusts them as it continuely learns are more and more examples.

### Shuffle the order of elements in a tensor 

#### why would you want to shuffle the elements of a tensor? 

Say we had 15,000 images of ramen and spaghetti, and the first 10,000 images were of ramen and the last 5,000 were of spaghetti. 

Now this order can affect how our neural network learns. Such as if we don't shuffle then our neural network would adjust its random weights too much so to ramen, because it is saying, well hey I only have to look at ramen. 


In [ ]:
# shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesnt effect learn)
not_shuffled = tf.constant([[10,7],
                           [3,4],
                           [2,5]])
not_shuffled.ndim 

#shuffle our nonshuffled tensor
tf.random.shuffle(not_shuffled)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [ ]:
#global level seed
tf.random.set_seed(42)
#operation level seed
tf.random.shuffle(not_shuffled, seed = 42) 

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

🛠️ ***Exercises*** Read through tensor flow documentation on random seed generation,
practice writing 5 random tensors and shuffle them. 

*** The interactions with operation-level seeds is as follows: *** 
1. If neither the global seed nor the operation seed is set: A randomly picked seed is used for this OP.

In [ ]:
#example
tf.random.shuffle(not_shuffled)
#this will create a random tensor with each run.

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [ 2,  5],
       [10,  7]], dtype=int32)>

2. If the flobal seed is set, but the operations eed is not: The system deterministically picks an 
operation seed in conjunction with the global seed so that it get a unique random sequence.
Within the same version of tensorflow and user code, this sequence is deterministic. However 
across different versions, this sequence ight change. If the code depends on a particular
seeds to work, specify both global and oepration-level seeds explicitly.  

In [ ]:
#example 
tf.random.set_seed(42) 
tf.random.shuffle(not_shuffled)
#for this we get different results with each call to the random OP 
#but the same sequence for every re-run program

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [ 2,  5],
       [10,  7]], dtype=int32)>

***note on what a seed is *** 
Seed function is used to save the state of a random function, so that it can generate same random numbers on multiple executions of the code on the same machine or on different machines

3. If the operation seed is set, but the global seed is not set: A default global seed and the 
specified operation seed are used to determine the random sequence

In [ ]:
tf.random.shuffle(not_shuffled, seed = 42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

4. If both the global and the operation seed are set: Both seeds are used in conjunction to
determine the random sequence 

In [ ]:
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed = 42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

**Other ways to make tensors**

In [ ]:
#Create a tensors of all ones 
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
#tensor of all zeros
tf.zeros(shape = (3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors 

The main difference between NumPy arrays and TensorFlow tensors is that
tensors can be run on GPU(much faster for numercial computing)

In [ ]:
#You can also turn NumPy arrays into tensors 
import numpy as np 
numpy_A = np.arange(1,25,dtype=np.int32)#create a numpy array between 1 and 25 
numpy_A


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

## Shape manipulation 

In [ ]:
#Turning that numpy_A data into a tensor 
#2 is the first dimension, then 3 look at the rows, then 4 cols
#in order for us to create the shape, we need to make sure it adds up to the
#value in the bottom array.
A = tf.constant(numpy_A, shape=(2,3,4))
B = tf.constant(numpy_A)

A,B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [ ]:
A.ndim

3

### Getting information from tensors
When dealing with tensors you probably want to be aware of the following attributes: 
* Shape 
* Rank 
* Axis or dimension 
* Size 

In [ ]:
# Create a rank 4 tensor 
#meaning the number of tensor dimenstions 
#4 dimensions 
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor 

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
#shape changes becasue we are getting the shape of the first set of 
#tensors
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
#Get varios attributes of our tensor 
print("Datatype of every element:", rank_4_tensor.dtype) 
print("number of dimensions: ", rank_4_tensor.ndim)
print("Shape of tensor:",rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0]) 
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))


Datatype of every element: <dtype: 'float32'>
number of dimensions:  4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)


Sometimes tensors are so big that we wont be able to just access them directly
so its important tat we can see the attributes of the tensors

### Indexing tensors 
Tensors can be indexed just like Python Lists. 

In [ ]:
# Get the first 2 elements of each dimensions 
rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
# get each dimension form each index except from the final one
#essentially flattens our values
rank_4_tensor[:1,:1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
#create a rank 2 tensor(2 dimension)
rank_2_tensor = tf.constant([[10,7],[3,4]]) 
rank_2_tensor.shape,rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [ ]:
# get the last item of each of our rank 2 tensor. 
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [ ]:
#helpful for when we need to line up our shapes for future video
#add in extra dimension to our rank 2 tensor 

rank_3_tensor = rank_2_tensor[...,tf.newaxis]
rank_3_tensor
#adds a dimension of one on the end

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
#Alternatice to tf.newaxis 
tf.expand_dims(rank_2_tensor, axis=-1)
#same as the above notation

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
#example if we expand the zero axis 
tf.expand_dims(rank_2_tensor,axis = 0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

In [ ]:
rank_2_tensor
#numbers stay the same but notice how their shapes change.

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### Manipulating tensors(tensor operations)
Basic operations

In [ ]:
# you can add values to a tensor using the addition operator

tensor = tf.constant([[10,7],
                      [3,4]])

tensor + 10 
#kind of works like a scalar KIND OF

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# original tensor is unchanged 
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
# multiplication 
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
#subtraction 
tensor - 10 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
# We can use the tensorflow built-in function too 
tf.multiply(tensor,10)
#saves gpu

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
tf.add(tensor,10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
tf.subtract(tensor,10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

Matrix multiplication

In [ ]:
print(tensor) 
tf.matmul(tensor,tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
# matrix multiplication with python operator @
tensor @ tensor 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

**The dot product** 

Matrix multiplication is also referred to as the dot product. 

you can perform matrix multiplication using: 

*`tf.matmul()`
*`tf.tesnordot()` 




In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed) 
#transposing is flipping the axis. 
x = tf.constant([[1,2],[3,4],[5,6]]) 
y = tf.constant([[7,8],[9,10],[11,12]]) 
x,y
tf.tensordot(tf.transpose(x),y, axes=1) 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between x and y (transposed) 
tf.matmul(x, tf.transpose(y))  

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between x and y (reshaped)

tf.matmul(x,tf.reshape(y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Check the values of Y, reshape y and transposed Y
print("Normal Y:")
print(y,"\n") 

print("Y reshaped to (2,3):")
print(tf.reshape(y,(2,3)),"\n")

print("y transposed :")
print(tf.transpose(y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2,3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

y transposed :
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


Generally, when performing matrix multiplication on two tensors, and one of he axes doesn't line up you will transpose rather than reshape one of the tensors so that you will satisify the multiplication rules

### Changing the datatype of a tensor (int32)

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7,7.4])
B.dtype

tf.float32

In [ ]:
C = tf.constant([7,10])
C.dtype 

tf.int32

In [ ]:
# Change from float32 to float16 (reduced precision)
#mixed precision
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

#### if you ever need to change the datatype of a tensor, because a calculuation isnt working right 
*`tf.cast()` 
is our friend

### Aggregating tensors 

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [ ]:
# Get the absolute values 
D = tf.constant([-7,-10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [ ]:
# Get absolute values 

tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Lets go through the following forms of aggregation 

* Get the minimum 
* Get the maximum 
* Get the mean of a tensor 
* Get the sum of a tensor 


In [ ]:
#Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0,100,size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([46, 27, 45, 26, 63, 29, 22, 73, 43, 78, 34,  4, 12,  0, 84,  1, 21,
       28, 25,  6, 69, 20, 46, 78, 77, 54, 30, 77, 25, 93, 33, 59, 51, 79,
       54, 81, 73, 41, 47, 88, 91,  5, 20, 10, 61, 45, 50, 74, 10, 15])>

In [ ]:
print("I am size", tf.size(E))
print("I am shape: ", E.shape)
print("I am ndim: ", E.ndim)

I am size tf.Tensor(50, shape=(), dtype=int32)
I am shape:  (50,)
I am ndim:  1


In [ ]:
# Find the minimum 
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [ ]:
#Find the maximum 
tf.reduce_max(E) 

<tf.Tensor: shape=(), dtype=int64, numpy=93>

In [ ]:
#finde the mean 
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=44>

In [ ]:
#find the sum 
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2223>

🛠️**Exercise:** With what We've just learned, find the variance and standard 
deviation of our 'E' tensor using Tensorflow methods. 

**What is variance?: ** 
Def: The average of the squered differences from the mean.

link: https://www.youtube.com/watch?v=x0rmUXWtSS8&ab_channel=TopTipBio 
Each variance value is squared which will not give us negative values. 

so: 
Mean 
variance = mean - datapoint 
Variance ^ 2 
then variance = variance / datapoint - 1

In [ ]:
#for variance 
import tensorflow_probability as tfp
tfp.stats.variance(E)


<tf.Tensor: shape=(), dtype=int64, numpy=731>

In [ ]:
#standard deviation is just the variance rooted 
#not as simple as you appear, this one we had to change the datatype for the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=27.033466>

## Find the positional Maximum and Minimum of tensor

"at which index of the tensor does the max or min occurr"

In [ ]:
# Create a new tensor for finding positional min and max 
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
#Find the positional Maximum 
#means position 42
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [ ]:
#index on or largest value position 
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
#check for quality 
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
#Find the positional minimum 
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [ ]:
#find the min using the positional min 
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [ ]:
#check quality
tf.reduce_min(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [ ]:
### Squeeze the tensor (removing all single dimension)
#shape is what we need to pay attention to when squeezing
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]),shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
#reducing the size of the tensor by 1
#puts it back into its essence
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>, TensorShape([50]))

### one-hot encode tensors 
one-hot is a form of encoding, think red, green, blue. 1 for red 0 for blue 0 for green. 
1 would just represent what is turned on 

In [ ]:
#Create a list of indices 
some_list = [0,1,2,3] # could be red, green, blue, purple 
depth = 4
# One hot encode our list of indices 
tf.one_hot(some_list,depth)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

### Squaring, log, square root 

In [ ]:
#Create a new tensor 
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
tf.math.sqrt(tf.cast(H,dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [ ]:
# Find the log
tf.math.log(tf.cast(H,dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and numpy

TensorFlow interacts perfectly with numPy arrays

In [ ]:
#Create a tensor directly from a numPy array
J = tf.constant(np.array([3.,7.,10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [ ]:
#convert our tensor back to a NumPY array
np.array(J),type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
#Convert tensor J to a NumPy array 
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3.,7.,10.]))
tensor_J = tf.constant([3.,7.,10.])
#check the data types of each 
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### What are regression problems? 
####Example regression problems 
How much will this house sell for? 

How many people will buy this app? 

How much will my health insurance be? 

How much should I save each week for fuel?

### What we are going to cover 
-architecture of a neural network regression model 

-input shapes and output shapes of a regression model(features and labels) 

-Creating custom data to view and fit

-steps in modelling 

-creating a model, compiling a model, fitting a mdoel, evaluating a model

-Different evaluation methods 

-saving and loading models


In [ ]:
### Regression inputs and outputs 
